In [1]:
import os
import pandas as pd
import xarray as xr
import sys
sys.path.append('C:/Users/flapet/OneDrive - NOC/Documents/utils_python')
from functions.float_download import *
import urllib3
import shutil
import requests

In [2]:
#wmo_number = [6990636, 3901581, 1902695, 4903659, 7902223, 1902637, 3901586, 1902304, 4903532]
wmo_number = [6904240, 7902223, 3901581, 6990636, 3902261, 3902258, 4903659, 3902260, 3901586, 1902304, 6901514, 6901515, 6904185, 4903365, 6901484]

In [3]:
root = 'C:/Users/flapet/OneDrive - NOC/Documents/IDAPro/lib/db_building'
profile_dir = root + '/data/argo_nc/'
natl_dir = profile_dir + 'North_Atlantic/'
nitrate_floats = profile_dir + 'Nitrate_floats/'

# Create GO-BGC folders if they do not exist yet
if 'data' not in os.listdir(root):
  os.mkdir(root + '/data')
if 'argo_nc' not in os.listdir(root + '/data'):
  os.mkdir(profile_dir)

In [4]:
wmoids, gdac_index, downloaded_filenames \
                   = argo_gdac(floats=wmo_number, save_to=nitrate_floats, overwrite_index=True, overwrite_profiles=True)

>>> File argo_synthetic-profile_index.txt already exists. Overwriting with new version.
>>> Successfully downloaded argo_synthetic-profile_index.txt.


C:\Users/flapet/OneDrive - NOC/Documents/utils_python\functions\float_download.py:107: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  gdac_index = pd.read_csv(save_to + index_filename,delimiter=',',header=8,parse_dates=['date','date_update'],


>>> File 1902304_Sprof.nc already exists. Overwriting with new version.
>>> Successfully downloaded 1902304_Sprof.nc.
>>> File 3902258_Sprof.nc already exists. Overwriting with new version.
>>> Successfully downloaded 3902258_Sprof.nc.
>>> File 3902260_Sprof.nc already exists. Overwriting with new version.
>>> Successfully downloaded 3902260_Sprof.nc.
>>> File 3902261_Sprof.nc already exists. Overwriting with new version.
>>> Successfully downloaded 3902261_Sprof.nc.
>>> File 4903365_Sprof.nc already exists. Overwriting with new version.
>>> Successfully downloaded 4903365_Sprof.nc.
>>> File 3901581_Sprof.nc already exists. Overwriting with new version.
>>> Successfully downloaded 3901581_Sprof.nc.
>>> File 3901586_Sprof.nc already exists. Overwriting with new version.
>>> Successfully downloaded 3901586_Sprof.nc.
>>> File 6904185_Sprof.nc already exists. Overwriting with new version.
>>> Successfully downloaded 6904185_Sprof.nc.
>>> File 4903659_Sprof.nc already exists. Overwriting wi

In [5]:
import polars as pl
import pyarrow.parquet as pq

In [6]:
variables = ['PLATFORM_NUMBER', 'N_PROF', 'JULD', 'LONGITUDE', 'LATITUDE', 'PRES', 'TEMP', 'PSAL', 'CHLA_ADJUSTED', 'CHLA_ADJUSTED_QC', 'BBP700_ADJUSTED', 'BBP700_ADJUSTED_QC', 'DOWNWELLING_PAR', 'DOWN_IRRADIANCE490', 'NITRATE_ADJUSTED', 'NITRATE_ADJUSTED_QC', 'DOXY_ADJUSTED']

df_list = []


for filename in [f for f in os.listdir(nitrate_floats) if os.path.isfile(os.path.join(nitrate_floats, f)) and f.endswith('.nc')]:
    filepath = os.path.join(nitrate_floats, filename)
    ds = xr.open_dataset(filepath)
    
    # Check for missing variables
    available_vars = set(ds.data_vars.keys())  # All variables in the current dataset
    missing_vars = [var for var in variables if var not in available_vars]
    
    # Add missing variables to the dataset with NaN values, matching the dimensions
    for var in missing_vars:
        if 'N_PROF' in ds.dims:  # If the dataset has the 'N_PROF' dimension
            ds[var] = (('N_PROF',), [np.nan] * ds.dims['N_PROF'])  # Create a variable with NaN values matching 'N_PROF'
        else:
            ds[var] = np.nan  # If no dimensions, add as a scalar (unlikely for most BGC-Argo data)

    # Convert xarray Dataset to Polars DataFrame
    df = pl.DataFrame(ds[variables].to_dataframe())  # Convert selected variables to a DataFrame
    
    # Cast float columns to Float64 for consistency
    float_cols = [col for col in df.columns if df[col].dtype in [pl.Float32, pl.Float64]]
    df = df.with_columns([pl.col(col).cast(pl.Float64) for col in float_cols])  # Ensure all floats are Float64
    
    # Convert PLATFORM_NUMBER to a string
    df = df.with_columns(pl.col("PLATFORM_NUMBER").cast(pl.Utf8))
    
    df_list.append(df)
    
# Concatenate all DataFrames into one
argo_table = pl.concat(df_list)

argo_table.write_parquet(root + '/data/argo_pq/biocarbon_nitrate_floats_table.parquet')

C:\Users\flapet\AppData\Local\Temp\ipykernel_28324\3396843694.py:17: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  ds[var] = (('N_PROF',), [np.nan] * ds.dims['N_PROF'])  # Create a variable with NaN values matching 'N_PROF'
C:\Users\flapet\AppData\Local\Temp\ipykernel_28324\3396843694.py:17: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  ds[var] = (('N_PROF',), [np.nan] * ds.dims['N_PROF'])  # Create a variable with NaN values matching 'N_PROF'
C:\Users\flapet\AppData\Local\Temp\ipykernel_28324\3396843694.py:17: FutureWarning: The return type of `Dataset.dims` will be changed to return a se

In [17]:
res = argo_table.group_by(['PLATFORM_NUMBER']).len()
print(res)

shape: (15, 2)
┌─────────────────┬────────┐
│ PLATFORM_NUMBER ┆ len    │
│ ---             ┆ ---    │
│ str             ┆ u32    │
╞═════════════════╪════════╡
│ 4903659         ┆ 101048 │
│ 6901514         ┆ 52611  │
│ 6901515         ┆ 108186 │
│ 6904185         ┆ 226270 │
│ 3902260         ┆ 19456  │
│ …               ┆ …      │
│ 6904240         ┆ 154088 │
│ 7902223         ┆ 28728  │
│ 4903365         ┆ 68370  │
│ 6901484         ┆ 87490  │
│ 1902304         ┆ 98512  │
└─────────────────┴────────┘
